In [ ]:
%matplotlib inline
from __future__ import print_function

import numpy as np
import torch
import data.data_cost as dt
import common_libs.utilities as ut
import models.train as tr
import models.graph_models as md
import models.losses as ls
from ithemal import ithemal_utils
import random
from tqdm import tqdm
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
full_data = dt.load_dataset('/home/ithemal/ithemal/learning/pytorch/inputs/embeddings/code_delim.emb', data_savefile='/home/ithemal/ithemal/learning/pytorch/saved/time_haswell_0131.data')

In [ ]:
def paths_of_block(self):
    def paths_of_instr(i, parents):
        new_parents = parents + [i]
        if i.children:
            return sum((paths_of_instr(c, new_parents) for c in i.children), [])
        else:
            return [new_parents]
    return sum((paths_of_instr(i, []) for i in self.find_roots()), [])

In [ ]:
z = np.array([len(paths_of_block(d.block)) for d in full_data.data])

In [ ]:
full_data.data[np.argmax(z)].block.draw()

In [ ]:
sns.distplot(z[z < 50])

In [ ]:
paths_of_block(full_data.data[0].block)

In [ ]:
full_data.data[0].block.draw()

In [ ]:
code_ids = {d.code_id for d in full_data.test if mem_free(d)}

In [ ]:
test_times = times[times['code_id'].isin(code_ids)]

iaca_times = test_times[test_times['kind'] == 'iaca']
iaca_preds = iaca_times.groupby('code_id').mean()

llvm_times = test_times[test_times['kind'] == 'llvm']
llvm_preds = llvm_times.groupby('code_id').mean()

actual_times = test_times[test_times['kind'] == 'actual']
actual_preds = actual_times.groupby('code_id').mean()
# actual_preds = actual_preds.loc[iaca_preds.index]

In [ ]:
print(((iaca_preds - actual_preds).abs() / actual_preds).mean()['time'])
print(((llvm_preds - actual_preds).abs() / actual_preds).mean()['time'])

In [ ]:
max(sum(1 + len(i.srcs) + len(i.dsts) for i in d.block.instrs) for d in data.data)

In [ ]:
haswell_data = full_data

In [ ]:
def sym_map(dataset):
    m = collections.defaultdict(list)
    for datum in dataset.raw_data:
        (code_token, timing, code_intel, code_id) = datum
        m[code_intel].append(datum)
    return m
haswell_sym_map = sym_map(haswell_data)
skylake_sym_map = sym_map(skylake_data)
nehalem_sym_map = sym_map(nehalem_data)

In [ ]:
for (name, dataset) in (
    ('haswell', haswell_sym_map),
    ('skylake', skylake_sym_map),
    ('nehalem', nehalem_sym_map)
):
    torch.save([
        random.choice(code) for code in dataset.values()
    ], '../saved/time_{}_0131.data'.format(name))

In [ ]:
full_data = dt.load_dataset('/home/ithemal/ithemal/learning/pytorch/inputs/embeddings/code_delim.emb', data_savefile='/home/ithemal/ithemal/learning/pytorch/saved/time_haswell_0113.data')
orig_full_test = full_data.test

In [ ]:
len(full_data.test)

In [ ]:
ablated_data = dt.load_dataset('/home/ithemal/ithemal/learning/pytorch/inputs/embeddings/code_delim.emb', data_savefile='/home/ithemal/ithemal/learning/pytorch/saved/time_haswell_0113.data')
orig_ablated_test = ablated_data.test

In [ ]:
ithemal_utils.ablate_data(
    ablated_data, 
    [
        ithemal_utils.EdgeAblationType.NO_EDGES,
        ithemal_utils.EdgeAblationType.ADD_LINEAR_EDGES,
    ],
    0,
)

In [ ]:
cnx = ut.create_connection()
times = pd.read_sql('SELECT time_id, code_id, time, kind FROM times WHERE arch=1', cnx).set_index('time_id')
legal_code_ids = set(times[times['kind'] == 'iaca'].code_id)

In [ ]:
codes = pd.read_sql('SELECT code_id, code_token, code_intel, program, rel_addr, config_id FROM code', ut.create_connection()).set_index('code_id')
codes['code_token'] = codes['code_token'].apply(lambda x: list(map(int, filter(bool, x.split(',')))))

In [ ]:
n_splits = codes['code_token'].apply(lambda x: x.count(-1))
codes['time'] = times[times['kind'] == 'actual'].groupby('code_id').mean()['time']

In [ ]:
dataset_code_ids = set(d.code_id for d in data.data)

In [ ]:
# single_instr_codes = codes[codes.index.isin(dataset_code_ids)]
# single_instr_codes = single_instr_codes[n_splits == 3]
# single_instr_codes['opcode'] = single_instr_codes['code_token'].apply(lambda x: x[0])
# single_instr_nomem_codes = single_instr_codes[~single_instr_codes['code_token'].apply(lambda x: any(op >= data.mem_start for op in x))]

nomem_codes = codes[codes.index.isin(dataset_code_ids)]
nomem_codes = nomem_codes[~nomem_codes['code_token'].apply(lambda x: any(op >= data.mem_start for op in x))]

# memfull_codes = codes[codes.index.isin(dataset_code_ids)]
# memfull_codes = memfull_codes[memfull_codes['code_token'].apply(lambda x: any(op >= data.mem_start for op in x))]
# # memfull_codes = memfull_codes[memfull_codes['code_token'].apply(lambda x: any(op == 5 for op in x))]
# memfull_codes = memfull_codes[~memfull_codes['code_token'].apply(lambda x: any(op == 5 for op in x))]

to_plot = nomem_codes

variabilities = to_plot.groupby('code_intel').agg({'time': ['mean', 'std']})['time']
variable_opcs = (variabilities['std'] / variabilities['mean']).dropna().sort_values()
sns.distplot(variable_opcs)

In [ ]:
q = variabilities.sort_values('std').dropna()
float(len(q[q['std'] < 1e-9])) / len(q)

In [ ]:
codes[codes['code_intel'] == 'vucomisd xmm14, xmm2\nmov    edx, 0x00000001\nsetp   al\ncmovnz eax, edx\ntest   al, al\n']

In [ ]:
z = codes[~codes.index.isin(set(times.code_id))]
q = z[z['code_intel'] != '']
lens = q['code_intel'].apply(lambda x: x.count('\n') - 1)
print(len(set(q['code_intel'])))
sns.distplot(lens)
# print(q.iloc[0])

In [ ]:
new_code_ids = {d.code_id for d in full_data.data}
iaca_code_ids = set(times[times['kind'] == 'iaca'].code_id)
mem_free_code_ids = {d.code_id for d in data.test if mem_free(d)}
l = (new_code_ids & iaca_code_ids & mem_free_code_ids)
idxs = [i for (i, d) in enumerate(data.test) if d.code_id in l]
print(lstm_errs[idxs].mean())
print(iaca_errs[idxs].mean())
print(llvm_errs[idxs].mean())

In [ ]:
codes['code_id'] = codes.index

In [ ]:
pred = {}
for rec in codes.groupby('code_intel').agg({'time': 'mean', 'code_id': set}).to_records():
    for cid in rec[1]:
        pred[cid] = rec[2]
pred = pd.DataFrame.from_dict(pred, orient='index', columns=['pred'])
pred.index = pred.index.rename('code_id')

In [ ]:
ac_times = times[times['kind'] == 'actual']
actuals = ac_times[ac_times['code_id'].isin(l)].groupby('code_id').mean()
actuals = actuals.loc[[d.code_id for d in data.test if d.code_id in l]]

In [ ]:
((actuals['time'] - pred['pred']).abs() / actuals['time']).dropna().mean()

In [ ]:
codes.loc[418511].time

In [ ]:
print(codes.loc[418511]['code_intel'])

In [ ]:
z = codes[codes.index.isin(set(times.code_id))]
q = z[z['code_intel'] != '']
lens = q['code_intel'].apply(lambda x: x.count('\n') - 1)
sns.distplot(lens)

In [ ]:
codes['code_token_str'] = codes['code_token'].apply(str)

In [ ]:
z = codes[codes.index.isin({d.code_id for d in data.data})]
print('num blocks: {}'.format(len(z)))
groups = z.groupby(['code_intel']).agg(['size'])
print('num unique blocks: {}'.format(len(groups)))
groups_with_dup = groups[groups > 1]
print('num groups w/ dup: {}'.format(len(groups_with_dup)))
groups_with_dup.sum()

In [ ]:
float(len(groups[groups<2])) / len(groups)

In [ ]:
sns.distplot(groups_with_dup[groups_with_dup < 100])

In [ ]:
code_ids = [1, 2, 3]


In [ ]:
for i in range(1, 100):
    print()
    print()
    print(groups.sort_values().iloc[i])
    print(groups.sort_values().index[i])

In [ ]:
len(set(times[times['kind'] == 'actual'].code_id))

In [ ]:
codes[codes['code_intel'] == 'lea    rdx, [rdi+0x10]\nmov    ecx, 0x00403850\ntest   rcx, rcx\n']

In [ ]:
opc = 846

m_codes = single_instr_nomem_codes[single_instr_nomem_codes['opcode'] == opc].sort_values('time').dropna()
sns.distplot(m_codes['time'])
m_codes

In [ ]:
variable_opcs

In [ ]:
model = md.GraphNN(
    256, 256, 1,
    False, False, True
)
model.set_learnable_embedding(mode='none', dictsize=max(full_data.word2id) + 1)
train = tr.Train(model, full_data, tr.PredictionType.REGRESSION, ls.mse_loss, 1)
_ = train.load_checkpoint('../saved/paper_haswell_dag_sgd-6.mdl')
actual, ithemal_dag_pred = train.validate('/tmp/res.log')

In [ ]:
model = md.GraphNN(
    256, 256, 1,
    False, False, True
)
model.set_learnable_embedding(mode='none', dictsize=max(ablated_data.word2id) + 1)
train = tr.Train(model, ablated_data, tr.PredictionType.REGRESSION, ls.mse_loss, 1)
_ = train.load_checkpoint('../saved/paper_haswell_lstm_sgd-6.mdl')
_, ithemal_lstm_pred = train.validate('/tmp/res.log')

In [ ]:
test_code_ids = [d.code_id for d in data.test]
def get_times_of_kind(kind):
    filt = (times['kind'] == kind) & times['code_id'].isin(test_code_ids)
    return (
        times[filt]
        .set_index('code_id')
        .time
        .loc[test_code_ids]
        .values
    )
iaca_preds = get_times_of_kind('iaca')
llvm_preds = get_times_of_kind('llvm')

In [ ]:
actual = np.array(actual)
ithemal_dag_pred = np.array(ithemal_dag_pred)
ithemal_lstm_pred = np.array(ithemal_lstm_pred)

In [ ]:


start = 0
end = 1000
bins = np.arange(start, end + 1, 20)

ithemal_lstm_heatmap, _, _ = np.histogram2d(actual, ithemal_lstm_pred, bins=[bins, bins])
ithemal_dag_heatmap, _, _ = np.histogram2d(actual, ithemal_dag_pred, bins=[bins, bins])
# iaca_heatmap, _, _ = np.histogram2d(actual, iaca_preds, bins=[bins, bins])
# llvm_heatmap, _, _ = np.histogram2d(actual, llvm_preds, bins=[bins, bins])

for b in range(len(bins) - 1):
    div = ithemal_lstm_heatmap[b, :].sum() or 1
    ithemal_lstm_heatmap[b, :] /= div
    ithemal_dag_heatmap[b, :] /= div

heatmap = ithemal_lstm_heatmap - ithemal_dag_heatmap

extreme = max(map(abs, (heatmap.T.max(), heatmap.T.min())))
# extreme = 1
extent = [start, end, start, end]
# lognorm = matplotlib.colors.LogNorm(vmin = 20, vmax = heatmap.T.max(), clip = False)
lognorm = None
clim = (-extreme, extreme)
# clim = None

y_str = 'Predicted Throughput'
x_str = 'Block Length'
cmap = plt.get_cmap('bwr')
# cmap.set_clim(-extreme, extreme)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)
plt.imshow(heatmap.T, cmap=cmap, norm=lognorm, extent=extent, origin='lower', clim=clim)
plt.plot(np.linspace(start, end, 1000), np.linspace(start, end, 1000), 'k--', alpha=0.2)
cbar = plt.colorbar()
plt.tick_params(labelsize=10)
cbar.ax.tick_params(labelsize=10)
ax.set_xlabel(x_str, fontsize=14)
ax.set_ylabel(y_str, fontsize=14)
ax.set_title('Haswell', fontsize=16)
plt.show()

In [ ]:
def mem_free(datum):
    for i in datum.block.instrs:
        if any(s >= data.mem_start for s in i.srcs + i.dsts):
            return False
        
    return True

In [ ]:
data = ablated_data

In [ ]:
n_mem_free = sum(1 for d in data.test if mem_free(d))
n_mem_full = sum(1 for d in data.test if mem_free(d))
print('mem free: {}, mem full: {}, total: {}, ratio: {}'.format(
    n_mem_free,
    n_mem_full,
    len(data.test),
    float(n_mem_free) / len(data.test)
))

In [ ]:
bad_blocks = [(i, d.block) for (i ,d) in enumerate(data.test) if 300 < d.y < 400 and mem_free(d)]

In [ ]:
len(bad_blocks)

In [ ]:
# 77/345 have 1 instr
sum(1 for (_, d) in bad_blocks if len(d.instrs) == 1)
# 76 are cmp (opc 172), 1 is test (opc 218)


In [ ]:
next(d for d in data.data for i in d.block.instrs if 'push' in str(i)).block.instrs[0].opcode

In [ ]:
import seaborn as sns

CMP = 172
TEST = 218
POP = 178
PUSH = 176

sns.distplot([d.y for d in data.data if len(d.block.instrs) == 1 and d.block.instrs[0].opcode in (POP,) and d.block.instrs[0].dsts[0] == 4])

In [ ]:
# q = next(d for d in data.data if len(d.block.instrs) == 1 and d.block.instrs[0].opcode in (POP,) and d.y > 800 and d.block.instrs[0].dsts[0] == 6)
# print(q.y)
# print(q.code_id)
# q.block.draw()

q = next(d for d in data.data if len(d.block.instrs) == 1 and d.block.instrs[0].opcode in (POP,) and d.y < 200 
         and d.block.instrs[0].dsts[0] == 4
        )
print(q.block.instrs[0].dsts[0])
print(q.y)
print(q.code_id)
q.block.draw()

In [ ]:
q.block.instrs[0].dsts


In [ ]:
import collections
z = collections.Counter()
for (_, d) in bad_blocks:
    if len(d.str) == 
    z[len(d.instrs)] += 1
print(z)

# CHARITH LOOK HERE

In [ ]:
bad_blocks = [
    (i, d.block) for (i, d) in enumerate(data.test) if 
    not mem_free(d) and 
    dag_errs[i] < 0.3 and 
    llvm_errs[i] > 0.6 and 
    600 < d.y < 1000
]
len(bad_blocks)

In [ ]:
times

In [ ]:
qa = [d.y for d in data.data if 
      len(d.block.instrs) == 1 
      and not mem_free(d) and 
#      all(5 not in i.srcs for i in d.block.instrs) and
#      all(5 not in i.dsts for i in d.block.instrs) and
         d.block.instrs[0].opcode == 176
#       and d.y > 100
    ]
sns.distplot(qa)

# RE-RUN THIS CELL TO SAMPLE

In [ ]:
idx, block = random.choice(bad_blocks)
print('Actual: {}, errs: DAG: {:.2f}, LSTM: {:.2f}, LLVM: {:.2f}, IACA: {:.2f}'.format(
    actual[idx],
    dag_errs[idx],
    lstm_errs[idx],
    llvm_errs[idx],
    iaca_errs[idx],
))
block.draw()

In [ ]:
q = next(b for b in data.data for i in b.block.instrs if ' ax' in str(i))
q.block.instrs[-1].srcs
q.block.draw()

In [ ]:
def errof(preds):
    return np.abs(actual - preds) / actual
dag_errs = errof(ithemal_dag_pred)
lstm_errs = errof(ithemal_lstm_pred)
iaca_errs = errof(iaca_preds)
llvm_errs = errof(llvm_preds)
block_lens = np.array([len(d.block.instrs) for d in data.test])

def plot_legal(legal, title):
    max_txput = 1000
    
    legal_idxs = [i for i in range(len(data.test)) if legal(data.test[i])]
    
    errs = pd.DataFrame.from_items(
        zip(actual, zip(dag_errs, lstm_errs, iaca_errs, llvm_errs)), 
        orient='index', 
        columns=['dag_errs', 'lstm_errs', 'iaca_errs', 'llvm_errs']
    )
    errs = errs.iloc[legal_idxs]
    errs = errs[errs.index <= max_txput]

    means = errs.groupby(pd.cut(errs.index, 50)).mean().dropna()
    
    mids = [i.mid for i in means.index]
    err_ax = plt.gca()
    err_ax.plot(mids, means.lstm_errs, label='LSTM errors')
    err_ax.plot(mids, means.dag_errs, label='DAG errors')
    err_ax.plot(mids, means.iaca_errs, label='IACA errors')
    err_ax.plot(mids, means.llvm_errs, label='LLVM errors')
    err_ax.set_ylabel('Average error')
    err_ax.set_xlabel('Measured BB throughput')
    err_ax.legend(loc=0)
    
    cdf_ax = err_ax.twinx()
    legal_actual = actual[legal_idxs]
    legal_block_lens = block_lens[legal_idxs]
    actuals_to_hist = legal_actual[legal_actual <= max_txput]
    vals, bins = np.histogram(actuals_to_hist, bins=1000)
    vals = vals.astype(np.float64)
    vals /= vals.sum()
    vals = vals.cumsum()
    mids = (bins[:-1] + bins[1:]) / 2
    cdf_ax.plot(mids, vals, 'k--', label='CDF of block frequency')
    cdf_ax.legend(loc=0)
    
    plt.title('{} ({:.2f}%, LSTM err: {:.3f}, DAG err: {:.3f})'.format(
        title, 
        100.0 * len(legal_block_lens) / len(data.test),
        lstm_errs.mean(),
        dag_errs.mean(),
    ))

plt.figure(figsize=(8, 6))
plot_legal(mem_free, 'Memory-free blocks')

plt.figure(figsize=(8, 6))
plot_legal(lambda x: not mem_free(x), 'Memory-containing blocks')
plt.plot()

In [ ]:
predictable_idxs = [
    i for (i, (_, r)) in enumerate(errs.iterrows()) if
#     r['dag_errs'] < .5 * r['lstm_errs'] and
#     r['dag_errs'] >= 0.05 and
    mem_free(data.test[i]) and
    actual[i] > 600 and actual[i] < 1000 and
    len(data.test[i].block.instrs) > 1 and
    True
]
len(predictable_idxs)

In [ ]:
r = random.choice(predictable_idxs)
print('Test index {}\n\n{}\n\n'.format(
    r,
    '\n'.join(map(str, data.test[r].block.instrs))
))
print('Actual: {}\nDag pred: {:.0f} ({:.3f} err)\nLSTM Pred: {:.0f} ({:.3f} err)'.format(
    actual[r],
    ithemal_dag_pred[r], errs.iloc[r]['dag_errs'],
    ithemal_lstm_pred[r], errs.iloc[r]['lstm_errs'],
))
data.test[r].block.draw()